In [ ]:
%%capture
!pip3 install torch>=1.3.0
!pip3 install subword-nmt &> log

!wget https://www.dropbox.com/s/qak4r5dtrktx5cu/data.txt?dl=0 -O data.txt # hotel data
!wget https://www.dropbox.com/s/wdc6jeaaprw0rey/ru-en-names-augment.txt?dl=0 -O ru-en-names-augment.txt # ru-en names augmented data
!wget https://www.dropbox.com/s/g1zqqhfb0jlw42h/vocab.py?dl=0 -O vocab.py
!wget https://www.dropbox.com/s/zu3vesqi107pph0/utils.py?dl=0 -O utils.py

In [ ]:
from nltk.tokenize import WordPunctTokenizer
from subword_nmt.learn_bpe import learn_bpe
from subword_nmt.apply_bpe import BPE

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
%matplotlib inline

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# BPE Tokenizing

In [ ]:
# Tokenizing & applying BPE rules

tokenizer = WordPunctTokenizer()
def tokenize(x):
    return ' '.join(tokenizer.tokenize(x.lower()))

# split and tokenize the data
with open('train.en', 'w') as f_src,  open('train.ru', 'w') as f_dst:
    for line in open('data.txt', 'r'):
        src_line, dst_line = line.strip().split('\t')
        f_src.write(tokenize(src_line) + '\n')
        f_dst.write(tokenize(dst_line) + '\n')


# build and apply bpe vocs
bpe = {}
for lang in ['en', 'ru']:
    learn_bpe(open('./train.' + lang), open('bpe_rules.' + lang, 'w'), num_symbols=8000)  # 1. learn_bpe rules
    bpe[lang] = BPE(open('./bpe_rules.' + lang))  # 2. create instance of BPE class
    
    with open('train.bpe.' + lang, 'w') as f_out:
        for line in open('train.' + lang):
            f_out.write(bpe[lang].process_line(line.strip()) + '\n')  # 3. apply BPE tokenization to our data        


# Building vocabularies

data_inp = np.array(open('./train.bpe.ru').read().split('\n'))
data_out = np.array(open('./train.bpe.en').read().split('\n'))

train_inp, dev_inp, train_out, dev_out = train_test_split(data_inp, data_out, test_size=3000,
                                                          random_state=42)
for i in range(3):
    print('inp:', train_inp[i])
    print('out:', train_out[i], end='\n\n')            

from vocab import Vocab

inp_voc = Vocab.from_lines(train_inp) # creates an instance of Vocab class from input lines (ru (input) vocab here)
out_voc = Vocab.from_lines(train_out) # en (output) vocab

print(f'Length of input (Russian) BPE vocabulary = {len(inp_voc)}')
print(f'Length of output (English) BPE vocabulary = {len(out_voc)}')

100%|██████████| 8000/8000 [00:09<00:00, 818.20it/s] 


inp: на территории обустроена бесплатная частная парковка .
out: free private parking is available on site .

inp: кроме того , в 5 минутах ходьбы работают многочисленные бары и рестораны .
out: guests can find many bars and restaurants within a 5 - minute walk .

inp: отель san mi@@ gu@@ el расположен в центре мор@@ ели@@ и , в 750 метрах от главной площади города и кафедрального собора .
out: hotel san miguel is located in central more@@ lia , 750 metres from the city ’ s main square and cathedral .

Length of input (Russian) BPE vocabulary = 8048
Length of output (English) BPE vocabulary = 7801


In [ ]:
!pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece] wandb

In [ ]:
import transformers

from transformers import AutoTokenizer
    
model_checkpoint = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, padding=False)

# Hotels data prep

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to('cuda') # 'cuda' 

In [ ]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, load_metric

def parse_ruencorp(train_inp=train_inp, dev_inp=dev_inp, train_out=train_out, dev_out=dev_out, mode='train'):
    if mode == 'train':
        for src_line, dst_line in zip(train_inp, train_out):
            yield {"translation": {"ru": src_line, "en": dst_line}}
    elif mode == 'test':
        for src_line, dst_line in zip(dev_inp, dev_out):
            yield {"translation": {"ru": src_line, "en": dst_line}}

In [ ]:
train_df = pd.DataFrame(parse_ruencorp(mode='train'))
test_df = pd.DataFrame(parse_ruencorp(mode='test'))

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

raw_datasets = DatasetDict({'train': train_dataset, 
                            'test': test_dataset})

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 47001
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
})

In [ ]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "ru"
target_lang = "en"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/47001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 47001
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
})

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    logging_strategy="steps",
    logging_steps=150,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    num_train_epochs=5,
    predict_with_generate=True,
    report_to="wandb",
)
    

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from datasets import load_metric
from torchmetrics.text.rouge import ROUGEScore
from nltk.translate.meteor_score import single_meteor_score
metric = load_metric("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
    
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    # Calculate BLEU score
    bleu = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": bleu["score"]}
    # Calculate METEOR
    meteor = np.mean([single_meteor_score(tokenizer.tokenize(ref[0]), tokenizer.tokenize(pred)) for ref, pred in zip(decoded_labels, decoded_preds)])
    result['meteor'] = meteor

    # Calculate ROUGE score
    rouge_score = ROUGEScore()
    rouge = rouge_score(decoded_preds, decoded_labels)
    # result["rouge"] = rouge
    for k,v in rouge.items():
      result[k] = v.item()

    # Calculate predicted lengths
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Inference & BLEU before fine-tuning

## Translations & BLEU before fine-tuning

In [ ]:
def translate(src_text):
    translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True).to('cuda'))
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated] 

In [ ]:
translate('Водохранилище Селиховское') # Плохо справляется с транслитерацией: Правильный ответ - Vodokhranilishche Selikhovskoye

['Selichovsky Water Reservoir']

In [ ]:
translate('поездка до аэропорта домодедово займет 50 минут .')

["It's gonna take 50 minutes to get to the airport."]

In [ ]:
translate('по утрам гостям сервируют сладкий завтрак « шведский стол »')

['In the morning, the guests are served with a sweet breakfast of the Swedish table.']

In [ ]:
translate('трехзвездочный отель idea piacenza расположен всего в 200 метрах от съезда')

['The three-star idea piacenza hotel is only 200 metres from the exit.']

In [ ]:
translate('до красивого средневекового города ротенбург - об - дер - таубер - всего 7 км .')

['To the beautiful medieval city of Rothenburg - o-der-tauber - only 7 km.']

In [ ]:
translate('расстояние до бангкока составляет 180 км .')

['The distance to the bancock is 180 km.']

In [ ]:
translate('на территории также имеется бассейн с большой солнечной террасой и принадлежности для барбекю .')

['The Territory also has a large solar terrace basin and barbecue supplies.']

In [ ]:
from tqdm import tqdm

references = []
predictions = []

with open('no-finetune-all_3k_eval_translations.txt','w') as f_trans:
    for inp_line, out_line in tqdm(zip(dev_inp, dev_out)):
        translated_inp = translate(inp_line)
        inp = inp_line.replace('@@ ', '')
        trans = translated_inp[0].replace('@@ ', '').replace('@@', '').replace('@', '').lower()
        predictions.append(trans)

        out = out_line.replace('@@ ', '')
        references.append(out)
        
        f_trans.write(f'input line: {inp}' + '\n')
        f_trans.write(f'translated line: {trans}' + '\n')
        f_trans.write(f'target line: {out}' + '\n')
        f_trans.write('\n')

3000it [22:07,  2.26it/s]


In [ ]:
!head -n 100 no-finetune-all_3k_eval_translations.txt

input line: в распоряжении гостей общая кухня и общая гостиная .
translated line: a common kitchen and a common living room are available to guests.
target line: a shared equipped kitchen and a common living room are provided to guests .

input line: на территории виллы shengsi huajing находится сад и терраса .
translated line: in villa shengsi huajing is the garden and terrace.
target line: at shengsi huajing villa you will find a garden and a terrace .

input line: расстояние от отеля libuše до ближайшей станции метро kobylisy ( линия с ), от которой можно добраться до центрального железнодорожного вокзала праги и центра города , составляет 500 метров .
translated line: the distance from the hotel libuše to the nearest subway station kobylisy (line c ) from which it is possible to reach the main railway station of praga and the city centre is 500 metres.
target line: the nearest metro station at kobylisy , on line c , is set 500 metres from hotel libuše , and it offers connections to

In [ ]:
!du -h no-finetune-all_3k_eval_translations.txt

1.2M	no-finetune-all_3k_eval_translations.txt


In [ ]:
pip install torchmetrics

In [ ]:
import sacrebleu

bleu_score = sacrebleu.corpus_bleu(predictions, [references])
print("BLEU score: ", bleu_score.score)

BLEU score:  11.613923846171362


In [ ]:
from nltk.translate.meteor_score import single_meteor_score
meteor_score = np.mean([single_meteor_score(tokenizer.tokenize(ref), tokenizer.tokenize(pred)) for ref, pred in zip(references, predictions)])
print("METEOR score: ", meteor_score)

0.6563934647583041

In [ ]:
rouge = rouge_score(predictions, references)

{'rouge1_fmeasure': tensor(0.4570),
 'rouge1_precision': tensor(0.4454),
 'rouge1_recall': tensor(0.4874),
 'rouge2_fmeasure': tensor(0.2017),
 'rouge2_precision': tensor(0.1969),
 'rouge2_recall': tensor(0.2155),
 'rougeL_fmeasure': tensor(0.3859),
 'rougeL_precision': tensor(0.3764),
 'rougeL_recall': tensor(0.4113),
 'rougeLsum_fmeasure': tensor(0.3900),
 'rougeLsum_precision': tensor(0.3804),
 'rougeLsum_recall': tensor(0.4156)}

In [ ]:
meteor_score([tokenizer.tokenize(references[0])], tokenizer.tokenize(predictions[0]))

0.6563934647583041

# Fine-tuning on ``hotel data``

In [ ]:
pip install wandb --upgrade

In [ ]:
import wandb

In [ ]:
wandb.init()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Bleu,Meteor,Rouge1 Fmeasure,Rouge1 Precision,Rouge1 Recall,Rouge2 Fmeasure,Rouge2 Precision,Rouge2 Recall,Rougel Fmeasure,Rougel Precision,Rougel Recall,Rougelsum Fmeasure,Rougelsum Precision,Rougelsum Recall,Gen Len
1000,1.440600,1.272107,32.569700,0.662100,0.611300,0.637300,0.600600,0.386300,0.404000,0.379200,0.540600,0.563500,0.531000,0.545000,0.568200,0.535300,26.730000
2000,1.262400,1.126208,34.962600,0.676000,0.628000,0.653000,0.617900,0.409400,0.426300,0.402700,0.559400,0.581200,0.550500,0.563900,0.586000,0.554800,26.653300
3000,1.117400,1.046594,37.322400,0.691800,0.636500,0.652200,0.634400,0.423700,0.434900,0.422400,0.568800,0.582400,0.567100,0.573900,0.587700,0.572000,27.277000
4000,1.060500,1.007228,38.172600,0.697500,0.648100,0.666100,0.643300,0.435500,0.448300,0.432200,0.582100,0.597900,0.577900,0.587300,0.603300,0.583000,27.138700
5000,1.019900,0.970884,38.645400,0.698200,0.649000,0.671000,0.641000,0.436600,0.451800,0.431500,0.580400,0.599400,0.573600,0.586400,0.605700,0.579400,26.919000
6000,0.965500,0.946799,39.200400,0.702500,0.652900,0.671700,0.647600,0.441900,0.455100,0.438500,0.585200,0.601400,0.580700,0.590300,0.606900,0.585600,27.106000
7000,0.929300,0.930782,39.380300,0.704100,0.654300,0.671300,0.650200,0.443900,0.455700,0.441300,0.585700,0.600500,0.582200,0.591700,0.606900,0.588000,26.977300
8000,0.901300,0.914293,40.126300,0.707900,0.661000,0.680300,0.655000,0.452900,0.466200,0.449300,0.594000,0.610600,0.589000,0.599600,0.616500,0.594300,26.952000
9000,0.876500,0.902407,40.499000,0.708200,0.660000,0.679000,0.654300,0.452200,0.465700,0.448300,0.593600,0.610000,0.588800,0.598900,0.615700,0.594000,27.041700
10000,0.886500,0.892970,40.470500,0.708300,0.661600,0.681900,0.654800,0.453800,0.468200,0.449300,0.594100,0.611700,0.588200,0.600000,0.617900,0.593800,26.897300


TrainOutput(global_step=14690, training_loss=1.012709327941853, metrics={'train_runtime': 6961.4023, 'train_samples_per_second': 33.758, 'train_steps_per_second': 2.11, 'total_flos': 5564188318629888.0, 'train_loss': 1.012709327941853, 'epoch': 5.0})

translations after fine-tuning

In [ ]:
translate('поездка до аэропорта домодедово займет 50 минут .')

['domodedovo airport is a 50 - minute drive away.']

In [ ]:
translate('по утрам гостям сервируют сладкий завтрак « шведский стол »')

['a sweet breakfast buffet is served every morning.']

In [ ]:
translate('трехзвездочный отель idea piacenza расположен всего в 200 метрах от съезда')

['the 3 - star idea pic@@ enza hotel is just 200 metres from the exit.']

In [ ]:
translate('до красивого средневекового города ротенбург - об - дер - таубер - всего 7 км .')

['the beautiful medieval town of rot@@ en@@ burg - o@@ der tab@@ er is just 7 km away.']

In [ ]:
translate('расстояние до бангкока составляет 180 км .')

['bangkok is 180 km away.']

In [ ]:
translate('на территории также имеется бассейн с большой солнечной террасой и принадлежности для барбекю .')

['there is also a swimming pool with a large sun terrace and barbecue facilities.']

In [ ]:
from tqdm import tqdm

references = []
predictions = []

with open('finetune-all_3k_eval_translations.txt','w') as f_trans:
    for inp_line, out_line in tqdm(zip(dev_inp, dev_out)):
        translated_inp = translate(inp_line)
        inp = inp_line.replace('@@ ', '')
        trans = translated_inp[0].replace('@@ ', '').replace('@@', '').replace('@', '').lower()
        predictions.append(trans)

        out = out_line.replace('@@ ', '')
        references.append(out)
        
        f_trans.write(f'input line: {inp}' + '\n')
        f_trans.write(f'translated line: {trans}' + '\n')
        f_trans.write(f'target line: {out}' + '\n')
        f_trans.write('\n')

3000it [15:23,  3.25it/s]


## Saving fine-tuned model weights to drive.google

In [ ]:
torch.save(model.state_dict(), 'fine-tuned-model.pt')

In [ ]:
import pickle

# Get the model weights as a dictionary
weights = model.state_dict()

# Save the weights as a pickle file
with open("transformer_weights_after_fine_tuning.pkl", "wb") as f:
    pickle.dump(weights, f)

### Augmentation JW300 En data

In [ ]:
!wget https://www.dropbox.com/s/yk5s1x5wf7xxfue/JW300-en-ru-train.tsv?dl=0 -O JW300-en-ru-data-augmentation.tsv # data for augmentation

--2023-04-25 06:10:47--  https://www.dropbox.com/s/yk5s1x5wf7xxfue/JW300-en-ru-train.tsv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/yk5s1x5wf7xxfue/JW300-en-ru-train.tsv [following]
--2023-04-25 06:10:48--  https://www.dropbox.com/s/raw/yk5s1x5wf7xxfue/JW300-en-ru-train.tsv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc25aa95f3f0d8991d41bb415cb3.dl.dropboxusercontent.com/cd/0/inline/B61SS6QSx1_l2yDoVtABC9mKuFC_Ktlj-HzdxJbenwmDOxv6GioVkE6wFq1kWJECiuPrymFlCc_UsaC7R7ITx3M5p8mAM6c-gVwBmVBWfY_7M0co3O7R-DlCvn88Cua136eIEaUJT2ew1S37F56c3kjqKxudrcriH-zz_ERq6nFt3g/file# [following]
--2023-04-25 06:10:48--  https://uc25aa95f3f0d8991d41bb415cb3.dl.dropboxusercontent.com/cd/0/inline/B61SS6QSx1_l2yDoVtABC9mKuFC_Ktlj-HzdxJbenwmDOxv

In [ ]:
import pandas as pd
augmentation_data = pd.read_csv('JW300-en-ru-data-augmentation.tsv', delimiter='\t', header=None, names=['en','ru'])
# pd.read_csv('ru-en-names-augment.txt', delimiter='\t', header=None, names=['ru','en'])

In [ ]:
translated_augmentation_data = []

In [ ]:
for sent in translated_augmentation_data.to_list():
  print(sent)

Streaming output truncated to the last 5000 lines.
Возможно, что инфляция лишила тебя привычки позволять себе кое - что вне очереди.
Может быть, в твоем распоряжении еще никогда не было « дополнительных » денег, или потребности твоей семьи важнее твоих собственных.
Некоторые женщины хотят ограничить свою работу по специальности, чтобы посвящать больше времени особой добровольной деятельности — что уже сделали многие христианки, чтобы стать полновременными проповедницами библейской благой вести.
Как можно тогда, несмотря на ограниченные средства, быть изящно одетой и чувствовать себя довольной?
Не робей, этого можно добиться.
Сначала обратимся к шкафу для одежды
Давай сначала сделаем инвентаризацию.
Лучше всего опорожнить весь шкаф.
Рассортируй одежду на две кучи: на любимые и на нелюбимые.
Любимые — это те платья, которые ты часто одеваешь, в которых ты чувствуешь себя хорошо и которые тебе идут, а нелюбимые — это те платья, которые или устарели или больше тебе не впору.
Если ты кое - 

KeyboardInterrupt: ignored

In [ ]:
for sent in augmentation_data.to_list():
  print(sent)

Streaming output truncated to the last 5000 lines.
It was not uncommon for children to have to work off the company - store debts inherited from their father.
Note, for example, a part of an editorial, appearing in a New York newspaper in 1872: “Sometimes generation after generation works to pay back debts begun by their grandfathers.
Those who have a few coins in their pockets earn them by menial labor after working long hours in the earth. ” So it was that with no other place to go and no money to leave, the miners became slaves to the mineowners.
Since child - labor laws were not then known, mine operators took advantage of young males, sending them into the mines at a very early age to work long hours in cramped spaces where only their small bodies would fit.
Some as young as five would work topside separating coal from the slate as it moved along on conveyor belts, their fingers and hands often crushed out of shape.
Others, exhausted from 14 hours of work, fell into the conveyors 

KeyboardInterrupt: ignored

In [ ]:
augmentation_data = augmentation_data.dropna()

In [ ]:
train_inp_augmented, dev_inp_augmented, train_out_augmented, dev_out_augmented = train_test_split(augmentation_data['ru'], 
                                                                                                  augmentation_data['en'],
                                                                                                  test_size=15000, 
                                                                                                  random_state=42)

In [ ]:
train_inp_augmented

148555    Один фермер жаловался: « Не деревья, а колы то...
647330              Теперь оно тоже должно формироваться *.
53071                                            Честность.
821852    Из любви Бог предоставил выкуп, открывающий пу...
786097                        [ Иллюстрация на странице 20]
                                ...                        
267976    Дети вскоре замечают, что животные спариваются...
380780    Родственники были встревожены и пытались воспр...
135414                                                 ПЕРУ
692379    Именно любовь к духовным братьям и сестрам поб...
125211    Как по телефону можно позвонить в другой конец...
Name: ru, Length: 979868, dtype: object

In [ ]:
train_out_augmented

148555    One grower lamented: “The trees are just spike...
647330                      These too must now be molded. *
53071                                             N., Japan
821852    Love moved God to provide a ransom that would ...
786097                                [ Picture on page 20]
                                ...                        
267976    The process of mating is soon noticed, followe...
380780    Our relatives were alarmed and began to oppose...
135414                                                ANDES
692379    Love for his spiritual brothers and sisters mo...
125211    Just as a telephone enables you to talk to som...
Name: en, Length: 979868, dtype: object

In [ ]:
dev_inp_augmented

831082             10 — Как Эдом был « истреблен навсегда »?
689117     « А слушающий меня [Божью мудрость] будет жить...
348215     Попробуй взглянуть на ситуацию так: представь,...
1020651                             Видения пророка Захарии.
426313     Затем Иегова начал „творить чудеса среди них“,...
                                 ...                        
519901     Они действительно принесли всю десятину в дом ...
874976            У него было пристрастие к табаку и гашишу.
849106     Одной сестре, которая активно служила Иегове о...
233903     ДЖОН, о котором упоминалось в первой статье, п...
544567     Она настаивала, что вы – коммунисты, о чем гов...
Name: ru, Length: 15000, dtype: object

In [ ]:
dev_out_augmented

831082     vs 10 ​ — How was Edom “cut off to time indefi...
689117     “ As for the one listening to [godly wisdom], ...
348215     Look at it this way: Suppose you’re going on a...
1020651             Chariots and a Crown Safeguard You, Oct.
426313     Jehovah then started to ‘ do wonderful things ...
                                 ...                        
519901     They have truly brought the whole tithe into t...
874976     He was addicted to tobacco and hashish, which ...
849106     For instance, a sister who served Jehovah acti...
233903     JOHN, mentioned in the preceding article, beca...
544567     She insisted that you are Communists, as the C...
Name: en, Length: 15000, dtype: object

**Back-translation**

In [ ]:
sample_0 = augmentation_data['en'][0]
sample_0

'World War I'

In [ ]:
reversed_model_checkpoint = "Helsinki-NLP/opus-mt-en-ru"
reversed_model = AutoModelForSeq2SeqLM.from_pretrained(reversed_model_checkpoint).to('cuda') # 'cuda' 

In [ ]:
reversed_model.translate(sample_0)

In [ ]:
def parse_ruencorp(train_inp=train_inp_augmented, dev_inp=dev_inp_augmented, train_out=train_out_augmented, dev_out=dev_out_augmented, mode='train'):
    if mode == 'train':
        for ru_sample, en_sample in zip(train_inp_augmented, train_out_augmented):
            yield {"translation": {"ru": ru_sample, "en": en_sample}}
    elif mode == 'test':
        for src_line, dst_line in zip(dev_inp_augmented, dev_out_augmented):
            yield {"translation": {"ru": src_line, "en": dst_line}}

In [ ]:
train_df = pd.DataFrame(parse_ruencorp(mode='train'))
test_df = pd.DataFrame(parse_ruencorp(mode='test'))

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

raw_datasets_augmentation = DatasetDict({'train': train_dataset, 
                                         'test': test_dataset})

In [ ]:
raw_datasets_augmentation

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 979868
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 15000
    })
})

In [ ]:
tokenized_datasets_augmentation = raw_datasets_augmentation.map(preprocess_function, batched=True)

In [ ]:
tokenized_datasets_augmentation

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 994868
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
})

In [ ]:
raw_datasets_augmentation['train'][0]

{'translation': {'en': 'World War I', 'ru': 'Первая мировая война'}}

## Dataloaders for augmentation

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-augmented-{source_lang}-to-{target_lang}",
    logging_strategy="steps",
    logging_steps=150,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    num_train_epochs=2,
    predict_with_generate=True,
    report_to="wandb",
)
    

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

## Fine-tuning on autgmented data

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets_augmentation["train"],
    eval_dataset=tokenized_datasets_augmentation["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# train on augmented (999k), validation on hotel descriptions (3k)
trainer.train()

Step,Training Loss,Validation Loss,Bleu,Meteor,Rouge1 Fmeasure,Rouge1 Precision,Rouge1 Recall,Rouge2 Fmeasure,Rouge2 Precision,Rouge2 Recall,Rougel Fmeasure,Rougel Precision,Rougel Recall,Rougelsum Fmeasure,Rougelsum Precision,Rougelsum Recall,Gen Len
1000,2.197200,1.664838,25.679900,0.647800,0.612900,0.632100,0.609500,0.394900,0.407700,0.393100,0.545900,0.562800,0.542900,0.550300,0.567500,0.546900,23.092000
2000,2.110100,1.897227,24.488100,0.640200,0.606700,0.626900,0.601400,0.384900,0.398100,0.381900,0.538900,0.556600,0.534400,0.543300,0.561400,0.538500,22.989300


Step,Training Loss,Validation Loss,Bleu,Meteor,Rouge1 Fmeasure,Rouge1 Precision,Rouge1 Recall,Rouge2 Fmeasure,Rouge2 Precision,Rouge2 Recall,Rougel Fmeasure,Rougel Precision,Rougel Recall,Rougelsum Fmeasure,Rougelsum Precision,Rougelsum Recall,Gen Len
1000,2.197200,1.664838,25.679900,0.647800,0.612900,0.632100,0.609500,0.394900,0.407700,0.393100,0.545900,0.562800,0.542900,0.550300,0.567500,0.546900,23.092000
2000,2.110100,1.897227,24.488100,0.640200,0.606700,0.626900,0.601400,0.384900,0.398100,0.381900,0.538900,0.556600,0.534400,0.543300,0.561400,0.538500,22.989300
3000,2.030400,2.018572,22.929900,0.631000,0.594500,0.615000,0.589200,0.366700,0.379800,0.363700,0.524900,0.543100,0.520000,0.529000,0.547600,0.523900,23.124300
4000,2.025900,2.127951,21.751000,0.624000,0.585600,0.607000,0.579500,0.354300,0.367800,0.350500,0.516400,0.535500,0.510600,0.520700,0.540200,0.514700,23.062000
5000,1.980300,2.198995,21.203500,0.617600,0.580000,0.599700,0.575800,0.350300,0.363400,0.347400,0.511500,0.529100,0.507500,0.515600,0.533600,0.511300,23.214000
6000,1.911100,2.299289,20.552100,0.614300,0.574100,0.593800,0.569800,0.340300,0.353000,0.337600,0.505500,0.523000,0.501500,0.509200,0.527100,0.505000,23.199700


KeyboardInterrupt: ignored

translations after augmenting

In [ ]:
translate('поездка до аэропорта домодедово займет 50 минут .')

['It takes 50 minutes to drive to Domodedadogo airport.']

In [ ]:
translate('по утрам гостям сервируют сладкий завтрак « шведский стол »')

['A sweet breakfast is served buffet buffet']

In [ ]:
translate('трехзвездочный отель idea piacenza расположен всего в 200 метрах от съезда')

['3 - star lea piacenza hotel is located only 200 meters [200 m] from the convention']

In [ ]:
translate('до красивого средневекового города ротенбург - об - дер - таубер - всего 7 км .')

['The beautiful medieval town of Rottenburg - ob - der - Tauber is only 7 km away.']

In [ ]:
translate('расстояние до бангкока составляет 180 км .')

['The distance to bangcock is 180 km.']

In [ ]:
translate('на территории также имеется бассейн с большой солнечной террасой и принадлежности для барбекю .')

['There is also a pool with a large sun terrace and barbecue facilities.']

In [ ]:
from tqdm import tqdm

references = []
predictions = []

with open('augmentation-all_3k_eval_translations.txt','w') as f_trans:
    for inp_line, out_line in tqdm(zip(dev_inp, dev_out)):
        translated_inp = translate(inp_line)
        inp = inp_line.replace('@@ ', '')
        trans = translated_inp[0].replace('@@ ', '').replace('@@', '').replace('@', '').lower()
        predictions.append(trans)

        out = out_line.replace('@@ ', '')
        references.append(out)
        
        f_trans.write(f'input line: {inp}' + '\n')
        f_trans.write(f'translated line: {trans}' + '\n')
        f_trans.write(f'target line: {out}' + '\n')
        f_trans.write('\n')

3000it [15:04,  3.32it/s]


# Fine-tuning on augmented ``ru-en-names_data``

## Data prep

In [ ]:
ru_en_names = pd.read_csv('ru-en-names-augment.txt', delimiter='\t', header=None, names=['ru','en'])
ru_en_names.head(10)

,ru,en
0,Явидово,Yavidovo
1,Ступнево,Stupnevo
2,Стукшино,Stukshino
3,Строевичи,Stroyevichi
4,Страна Советов,Strana Sovetov
5,Урочище Ставрово,Urochishche Stavrovo
6,Старое Малиново,Staroye Malinovo
7,Старое Китово,Staroye Kitovo
8,Сотский,Sotskiy
9,Слапихино,Slapikhino


In [ ]:
ru_names = ru_en_names.ru.to_list()
en_names = ru_en_names.en.to_list()

In [ ]:
print(ru_names[:5])
print(en_names[:5])

['Явидово', 'Ступнево', 'Стукшино', 'Строевичи', 'Страна Советов']
['Yavidovo', 'Stupnevo', 'Stukshino', 'Stroyevichi', 'Strana Sovetov']


In [ ]:
from datasets import load_metric
metric = load_metric("sacrebleu")

from datasets import Dataset, DatasetDict
from datasets import load_dataset, load_metric

def parse_ruencorp(train_inp=ru_names, dev_inp=dev_inp, train_out=en_names, dev_out=dev_out, mode='train'):
    if mode == 'train':
        for ru_name, en_name in zip(ru_names, en_names):
            yield {"translation": {"ru": ru_name, "en": en_name}}
    elif mode == 'test':
        for src_line, dst_line in zip(dev_inp, dev_out):
            yield {"translation": {"ru": src_line, "en": dst_line}}

In [ ]:
train_df = pd.DataFrame(parse_ruencorp(mode='train'))
test_df = pd.DataFrame(parse_ruencorp(mode='test'))

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

raw_datasets_augmentation = DatasetDict({'train': train_dataset, 
                                         'test': test_dataset})

In [ ]:
raw_datasets_augmentation

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 187459
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
})

In [ ]:
raw_datasets_augmentation['train'][0]

{'translation': {'en': 'Yavidovo', 'ru': 'Явидово'}}

In [ ]:
# check len distribution after 

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "ru"
target_lang = "en"
def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples['translation']]
    targets = [ex[target_lang] for ex in examples['translation']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets_augmented = raw_datasets_augmentation.map(preprocess_function, batched=True)

Map:   0%|          | 0/187459 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets_augmented

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 187459
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
})

## Dataloaders for fine-tuning

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    logging_strategy="steps",
    logging_steps=150,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    num_train_epochs=2,
    predict_with_generate=True,
    report_to="wandb",
)
    

PyTorch: setting up devices


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
    
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Fine-tuning on autgmented data

In [ ]:
translate('Водохранилище Селиховское')

Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



['the sil@@ kh@@ ov@@ skaya water storage room']

In [ ]:
ru_names[:5]

['Явидово', 'Ступнево', 'Стукшино', 'Строевичи', 'Страна Советов']

In [ ]:
translate(ru_names[:5])

['I can see it.',
 'Stopnevo',
 'Stukchino',
 'Strojevichs',
 'Country of the Councils']

In [ ]:
ru_names[-5:]

['Озеро Полевое', 'Урочище Битюг', 'Гнилушка', 'Урочище Деево', 'Аркаим']

In [ ]:
translate(ru_names[-5:])

['Field Lake',
 'The Beatug Beautiful',
 "You're a rotten bastard.",
 "Dejevo's Daylight.",
 'Arkaim']

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets_augmented["train"],
    eval_dataset=tokenized_datasets_augmented["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 187459
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 23434
  Number of trainable parameters = 76147712
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Bleu,Gen Len
6000,0.340700,2.899605,11.826900,26.522300


The following columns in the evaluation set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 16
Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,

KeyboardInterrupt: ignored

In [ ]:
for inp_line, out_line in tqdm(zip(dev_inp[:500], dev_out[:500])):
    translated_inp = translate(inp_line)
    inp = inp_line.replace('@@ ', '')
    trans = translated_inp[0].replace('@@ ', '').replace('@@', '').replace('@', '').lower()
    predictions.append(trans)

    out = out_line.replace('@@ ', '')
    references.append(out)
    
    print(f'input line: {inp}' + '\n')
    print(f'translated line: {trans}' + '\n')
    print(f'target line: {out}' + '\n')
    print()
    # f_trans.write(f'input line: {inp}' + '\n')
    # f_trans.write(f'translated line: {trans}' + '\n')
    # f_trans.write(f'target line: {out}' + '\n')
    # f_trans.write('\n')

0it [00:00, ?it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

1it [00:00,  1.30it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



input line: в распоряжении гостей общая кухня и общая гостиная .

translated line: ostanovochnyy punkt punkt

target line: a shared equipped kitchen and a common living room are provided to guests .




2it [00:01,  1.13it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



input line: на территории виллы shengsi huajing находится сад и терраса .

translated line: villy shengsi i huazhing features a garden and terrasa.

target line: at shengsi huajing villa you will find a garden and a terrace .




3it [00:02,  1.05s/it]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



...

100it [00:43,  3.44it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



input line: в числе прочих удобств — dvd - плеер и телевизор с плоским экраном .

translated line: ostanovochnyy punkt dvd-player and a flat - screen tv.

target line: each comes with a dvd player and a flat - screen tv .




101it [00:44,  3.04it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

102it [00:44,  3.56it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

102it [00:44,  2.29it/s]

input line: гости могут посещать сезонный открытый бассейн .

translated line: ostanovochnyy punkt ostanovochnyy punkt ostanovochnyy punkt

target line: other facilities at pearl one include a seasonal outdoor pool .


input line: за несколько минут от отеля можно дойти до множества разных мест развлечений .

translated line: a variety of activities can be reached within minutes of the hotel.

target line: a variety of entertainment options can be explored in a walking distance of the hotel .




KeyboardInterrupt: ignored

In [ ]:
from tqdm import tqdm

references = []
predictions = []

with open('augmented-and-fine-tuned-all_3k_eval_translations.txt','w') as f_trans:
    for inp_line, out_line in tqdm(zip(dev_inp[:500], dev_out[:500])):
        translated_inp = translate(inp_line)
        inp = inp_line.replace('@@ ', '')
        trans = translated_inp[0].replace('@@ ', '').replace('@@', '').replace('@', '').lower()
        predictions.append(trans)

        out = out_line.replace('@@ ', '')
        references.append(out)
        
        f_trans.write(f'input line: {inp}' + '\n')
        f_trans.write(f'translated line: {trans}' + '\n')
        f_trans.write(f'target line: {out}' + '\n')
        f_trans.write('\n')

0it [00:00, ?it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

1it [00:00,  3.70it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

2it [00:00,  3.80it/s]Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}

3it [00:00,  3.01it/s]Generate config GenerationConfig {
  "ba

KeyboardInterrupt: ignored

In [ ]:
translate('Водохранилище Селиховское')

Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



['Vodokhranilishche Selikhovskoye']

In [ ]:
ru_names[:5]

['Явидово', 'Ступнево', 'Стукшино', 'Строевичи', 'Страна Советов']

In [ ]:
translate(ru_names[:5])

Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



['Yavidovo',
 'Stupnevo',
 'Stukshino',
 'Stroyevichi',
 'Ostanovochnyy Punkt Sovetov']

In [ ]:
ru_names[-5:]

['Озеро Полевое', 'Урочище Битюг', 'Гнилушка', 'Урочище Деево', 'Аркаим']

In [ ]:
translate(ru_names[-5:])

Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



['Ozero Polevoye',
 'Urochishche Bityug',
 'Gnilushka',
 'Urochishche Deyevo',
 'Arkaim']

In [ ]:
translate('на проспекте кирова')

Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



['On Profsektoye Kirova']

In [ ]:
translate('расстояние до города тамбор составляет 44 км')

Generate config GenerationConfig {
  "bad_words_ids": [
    [
      62517
    ]
  ],
  "bos_token_id": 0,
  "decoder_start_token_id": 62517,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "max_length": 512,
  "num_beams": 6,
  "pad_token_id": 62517,
  "transformers_version": "4.26.1"
}



['Razvaliny Tambar 44 km']

# Helpful funtions

save model weights as pickle file (and upload it to your disk)

In [ ]:
import pickle

# Get the model weights as a dictionary
weights = model.state_dict()

# Save the weights as a pickle file
with open("model_weights_no_fine_tuning.pkl", "wb") as f:
    pickle.dump(weights, f)

load .pickle weights from disk.google

In [ ]:

# Load the saved weights
with open("model_weights.pkl", "rb") as f:
    saved_weights = pickle.load(f)

# Set the model weights to saved weights
model.load_state_dict(saved_weights)

# Use the loaded model for inference

mount drive.google

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/NMT/

/content/drive/MyDrive/NMT


In [ ]:
%ls

'=1.3.0'                        checkpoint-10000/   train.bpe.en   utils.py
 all_3k_eval_translations.txt   data.txt            train.bpe.ru   vocab.py
 bpe_rules.en                   log                 train.en       wandb/
 bpe_rules.ru                   __pycache__/        train.ru


In [ ]:
!cp -r /content/opus-mt-ru-en-finetuned-ru-to-en/checkpoint-10000 "/content/drive/MyDrive/NMT/"

In [ ]:
!du -h checkpoint-10000

877M	checkpoint-10000


get model weights and check its memory allocation

In [ ]:
import pickle

# Get the model weights as a dictionary
weights = model.state_dict()

# Save the weights as a pickle file
with open("model_weights_no_fine_tuning.pkl", "wb") as f:
    pickle.dump(weights, f)

In [ ]:
%ls

'=1.3.0'        data.txt       train.bpe.en   train.ru
 bpe_rules.en   log            train.bpe.ru   vocab.py
 bpe_rules.ru   __pycache__/   train.en


In [ ]:
!du -h model_weights.pkl

du: cannot access 'model_weights.pkl': No such file or directory


In [ ]:
model.parameters